# KoBERT finetuning

### settings

In [ ]:
# !pip install ipywidgets  # for vscode
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import pandas as pd

In [ ]:
seed = 0
max_len = 64
batch_size = 260
warmup_ratio = 0.1
num_epochs = 1000
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
from kobert import get_tokenizer
from kobert import get_pytorch_kobert_model

In [ ]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
bertmodel, vocab = get_pytorch_kobert_model(cachedir=".cache")

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


### load data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
label_dict = {"유형":{"사실형":0, "추론형":1, "대화형":2, "예측형":3}, "극성":{"긍정":0, "부정":1, "미정":2}, "시제":{"과거":0, "현재":1, "미래":2}, "확실성":{"확실":0, "불확실":1}}

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/kobert/train.csv')

In [ ]:
for label in label_dict:
  for i in range(len(df)):
    df[label].iloc[i] = label_dict[label][df[label].iloc[i]]

In [ ]:
df

,ID,문장,유형,극성,시제,확실성,label
0,TRAIN_00000,0.75%포인트 금리 인상은 1994년 이후 28년 만에 처음이다.,0,0,1,0,사실형-긍정-현재-확실
1,TRAIN_00001,이어 ＂앞으로 전문가들과 함께 4주 단위로 상황을 재평가할 예정＂이라며 ＂그 이전이...,0,0,0,0,사실형-긍정-과거-확실
2,TRAIN_00002,정부가 고유가 대응을 위해 7월부터 연말까지 유류세 인하 폭을 30%에서 37%까지...,0,0,2,0,사실형-긍정-미래-확실
3,TRAIN_00003,"서울시는 올해 3월 즉시 견인 유예시간 60분을 제공하겠다고 밝혔지만, 하루 만에 ...",0,0,0,0,사실형-긍정-과거-확실
4,TRAIN_00004,익사한 자는 사다리에 태워 거꾸로 놓고 소금으로 코를 막아 가득 채운다.,0,0,1,0,사실형-긍정-현재-확실
...,...,...,...,...,...,...,...
16536,TRAIN_16536,"＇신동덤＇은 ＇신비한 동물사전＇과 ＇해리 포터＇ 시리즈를 잇는 마법 어드벤처물로, ...",0,0,0,0,사실형-긍정-과거-확실
16537,TRAIN_16537,"수족냉증은 어릴 때부터 심했으며 관절은 어디 한 곳이 아니고 목, 어깨, 팔꿈치, ...",0,0,0,0,사실형-긍정-과거-확실
16538,TRAIN_16538,김금희 소설가는 ＂계약서 조정이 그리 어려운가 작가를 격려한다면서 그런 문구 하나 ...,0,0,0,0,사실형-긍정-과거-확실
16539,TRAIN_16539,1만명이 넘는 방문자수를 기록한 이번 전시회는 총 77개 작품을 넥슨 사옥을 그대로...,0,0,0,1,사실형-긍정-과거-불확실


In [ ]:
train, val, _, _ = train_test_split(df, df['label'], test_size=0.2, random_state=seed, stratify = df["극성"])

In [ ]:
print(train["유형"].value_counts(), train["시제"].value_counts(), train["확실성"].value_counts(), sep="\n")

0    10857
1     1715
2      460
3      200
Name: 유형, dtype: int64
0    6402
1    5530
2    1300
Name: 시제, dtype: int64
0    12160
1     1072
Name: 확실성, dtype: int64


In [ ]:
print(val["유형"].value_counts(), val["시제"].value_counts(), val["확실성"].value_counts(), sep="\n")

0    2701
1     436
2     115
3      57
Name: 유형, dtype: int64
0    1630
1    1336
2     343
Name: 시제, dtype: int64
0    3032
1     277
Name: 확실성, dtype: int64


### define model

In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, bert_tokenizer, max_len,
                 pad, pair, train):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        self.train=train
        self.sentences = [transform([i]) for i in dataset["문장"]]
        if train:
          self.type_labels = [np.int32(i) for i in dataset["유형"]]
          self.polarity_labels = [np.int32(i) for i in dataset["극성"]]
          self.tense_labels = [np.int32(i) for i in dataset["시제"]]
          self.certainty_labels = [np.int32(i) for i in dataset["확실성"]]

    def __getitem__(self, i):
        if self.train:
          return (self.sentences[i] + (self.type_labels[i], self.polarity_labels[i], self.tense_labels[i], self.certainty_labels[i]))
        return (self.sentences[i],)

    def __len__(self):
        return (len(self.sentences))


In [ ]:
data_train = BERTDataset(train, tok, max_len, True, False, True)
data_val = BERTDataset(val, tok, max_len, True, False, True)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, shuffle=True, num_workers=0, drop_last=True)
val_dataloader = torch.utils.data.DataLoader(data_val, batch_size=batch_size, shuffle=False, num_workers=0)

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.type_classifier = nn.Linear(hidden_size , 4)
        self.polarity_classifier = nn.Linear(hidden_size , 3)
        self.tense_classifier = nn.Linear(hidden_size , 3)
        self.certainty_classifier = nn.Linear(hidden_size , 2)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        else:
            out = pooler
        return self.type_classifier(out), self.polarity_classifier(out), self.tense_classifier(out), self.certainty_classifier(out)

In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [ ]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

### train

In [ ]:
def validation(model, val_loader, criterion, device):
    model.eval()
    val_loss = []
    
    type_preds, polarity_preds, tense_preds, certainty_preds = [], [], [], []
    type_labels, polarity_labels, tense_labels, certainty_labels = [], [], [], []
    
    
    with torch.no_grad():
        for batch_id, (token_ids, valid_length, segment_ids, type_label, polarity_label, tense_label, certainty_label) in tqdm(enumerate(val_loader), total=len(val_loader)):
            token_ids = token_ids.long().to(device)
            segment_ids = segment_ids.long().to(device)
            valid_length = valid_length
            type_label = type_label.long().to(device)
            polarity_label = polarity_label.long().to(device)
            tense_label = tense_label.long().to(device)
            certainty_label = certainty_label.long().to(device)
            type_logit, polarity_logit, tense_logit, certainty_logit = model(token_ids, valid_length, segment_ids)
            
            loss = 0.25 * criterion['type'](type_logit, type_label) + \
                        0.25 * criterion['polarity'](polarity_logit, polarity_label) + \
                        0.25 * criterion['tense'](tense_logit, tense_label) + \
                        0.25 * criterion['certainty'](certainty_logit, certainty_label)
            
            val_loss.append(loss.item())
            
            type_preds += type_logit.argmax(1).detach().cpu().numpy().tolist()
            type_labels += type_label.detach().cpu().numpy().tolist()
            
            polarity_preds += polarity_logit.argmax(1).detach().cpu().numpy().tolist()
            polarity_labels += polarity_label.detach().cpu().numpy().tolist()
            
            tense_preds += tense_logit.argmax(1).detach().cpu().numpy().tolist()
            tense_labels += tense_label.detach().cpu().numpy().tolist()
            
            certainty_preds += certainty_logit.argmax(1).detach().cpu().numpy().tolist()
            certainty_labels += certainty_label.detach().cpu().numpy().tolist()
    
    type_f1 = f1_score(type_labels, type_preds, average='weighted')
    polarity_f1 = f1_score(polarity_labels, polarity_preds, average='weighted')
    tense_f1 = f1_score(tense_labels, tense_preds, average='weighted')
    certainty_f1 = f1_score(certainty_labels, certainty_preds, average='weighted')
    
    return np.mean(val_loss), type_f1, polarity_f1, tense_f1, certainty_f1

In [ ]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):

  log_df = {"train_losses":[], "val_losses":[], "type_f1s":[], "polarity_f1s":[], "tense_f1s":[], "certainty_f1s":[]}

  for e in range(num_epochs):

      criterion = {
          'type' : nn.CrossEntropyLoss().to(device),
          'polarity' : nn.CrossEntropyLoss().to(device),
          'tense' : nn.CrossEntropyLoss().to(device),
          'certainty' : nn.CrossEntropyLoss().to(device)
      }

      model.train()
      train_loss = []
      for batch_id, (token_ids, valid_length, segment_ids, type_labels, polarity_labels, tense_labels, certainty_labels) in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
          optimizer.zero_grad()
          token_ids = token_ids.long().to(device)
          segment_ids = segment_ids.long().to(device)
          valid_length = valid_length
          type_labels = type_labels.long().to(device)
          polarity_labels = polarity_labels.long().to(device)
          tense_labels = tense_labels.long().to(device)
          certainty_labels = certainty_labels.long().to(device)
          out = model(token_ids, valid_length, segment_ids)

          loss = 0.25 * criterion['type'](out[0], type_labels) + \
                      0.25 * criterion['polarity'](out[1], polarity_labels) + \
                      0.25 * criterion['tense'](out[2], tense_labels) + \
                      0.25 * criterion['certainty'](out[3], certainty_labels)
          loss.backward()
          torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
          optimizer.step()
          scheduler.step()  # Update learning rate schedule
          train_loss.append(loss.item())

      model.eval()
      val_loss, type_f1, polarity_f1, tense_f1, certainty_f1 = validation(model, val_loader, criterion, device)
      print(f'Epoch : [{e}] Train Loss : [{np.mean(train_loss):.5f}] Val Loss : [{val_loss:.5f}] 유형 F1 : [{type_f1:.5f}] 극성 F1 : [{polarity_f1:.5f}] 시제 F1 : [{tense_f1:.5f}] 확실성 F1 : [{certainty_f1:.5f}]')
      
      log_df["val_losses"].append(val_loss)
      log_df["train_losses"].append(np.mean(train_loss))
      log_df["type_f1s"].append(type_f1)
      log_df["polarity_f1s"].append(polarity_f1)
      log_df["tense_f1s"].append(tense_f1)
      log_df["certainty_f1s"].append(certainty_f1)

      pd.DataFrame(log_df).to_csv('/content/drive/MyDrive/kobert/saved_models/log.csv')
      torch.save(model.state_dict(), f"/content/drive/MyDrive/kobert/saved_models/model_state_dict{str(e+1).zfill(4)}.pth")

  return model

In [ ]:
infer_model = train(model, optimizer, train_dataloader, val_dataloader, scheduler, device)

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

Epoch : [0] Train Loss : [1.14653] Val Loss : [1.08070] 유형 F1 : [0.19067] 극성 F1 : [0.03026] 시제 F1 : [0.35150] 확실성 F1 : [0.56790]


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

Epoch : [30] Train Loss : [0.07658] Val Loss : [0.28231] 유형 F1 : [0.87850] 극성 F1 : [0.97392] 시제 F1 : [0.87904] 확실성 F1 : [0.92522]


  0%|          | 0/50 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

## Inference

In [ ]:
pretrained_dict = torch.load(r'/content/drive/MyDrive/kobert/saved_models/model_state_dict0018.pth')
model.load_state_dict(pretrained_dict)
infer_model = model

In [ ]:
data_test = BERTDataset(val, tok, max_len, True, False, train=False)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, shuffle=False, num_workers=0)

# test = pd.read_csv('/content/drive/MyDrive/kobert/test.csv')

# data_test = BERTDataset(test, tok, max_len, True, False, train=False)
# test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, shuffle=False, num_workers=0)

In [ ]:
def inference(infer_model, test_dataloader, device):
    infer_model.to(device)
    infer_model.eval()
    
    type_preds, polarity_preds, tense_preds, certainty_preds = [], [], [], []
    
    with torch.no_grad():
      for batch_id, i in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
        token_ids, valid_length, segment_ids = i[0][0], i[0][1], i[0][2]
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        type_logit, polarity_logit, tense_logit, certainty_logit = model(token_ids, valid_length, segment_ids)
        
        type_preds += type_logit.argmax(1).detach().cpu().numpy().tolist()
        polarity_preds += polarity_logit.argmax(1).detach().cpu().numpy().tolist()
        tense_preds += tense_logit.argmax(1).detach().cpu().numpy().tolist()
        certainty_preds += certainty_logit.argmax(1).detach().cpu().numpy().tolist()
        
    return type_preds, polarity_preds, tense_preds, certainty_preds

In [ ]:
type_preds, polarity_preds, tense_preds, certainty_preds = inference(infer_model, test_dataloader, device)

  0%|          | 0/13 [00:00<?, ?it/s]

In [ ]:
label_dict = {"유형":["사실형", "추론형", "대화형", "예측형"], "극성":["긍정", "부정", "미정"], "시제":["과거", "현재", "미래"], "확실성":["확실", "불확실"]}

In [ ]:
type_preds = [label_dict["유형"][x] for x in type_preds]
polarity_preds = [label_dict["극성"][x] for x in polarity_preds]
tense_preds = [label_dict["시제"][x] for x in tense_preds]
certainty_preds = [label_dict["확실성"][x] for x in certainty_preds]

In [ ]:
predictions = []
for type_pred, polarity_pred, tense_pred, certainty_pred in zip(type_preds, polarity_preds, tense_preds, certainty_preds):
    predictions.append(type_pred+'-'+polarity_pred+'-'+tense_pred+'-'+certainty_pred)

In [ ]:
# print(len(val["문장"]), len(polarity_preds))
# pd.DataFrame({"sentence":val["문장"], "유형":[label_dict["유형"][x] for x in val["유형"]], "극성":[label_dict["극성"][x] for x in val["극성"]], "시제":[label_dict["시제"][x] for x in val["시제"]], "확실성":[label_dict["확실성"][x] for x in val["확실성"]], "type_preds":type_preds, "polarity_preds":polarity_preds, "tense_preds":tense_preds, "certainty_preds":certainty_preds, "predictions":predictions}).to_csv("./prediction.csv")

## Submission

In [ ]:
submit = pd.read_csv('/content/drive/MyDrive/kobert/sample_submission.csv')
submit['label'] = predictions

ValueError: ignored

In [ ]:
submit.head()

In [ ]:
submit.to_csv('/content/drive/MyDrive/kobert/submissions/submission.csv', index=False)